In [63]:
import os
# Twitter API 
import tweepy

import requests

import configparser
# read config
config = configparser.ConfigParser()
config.read('../config.ini')

# Access API secrets in .env file
%load_ext dotenv
%dotenv

import pandas as pd

The dotenv extension is already loaded. To reload it, use:
  %reload_ext dotenv


In [45]:
api_key = config['twitter']['api_key']
api_key_secret = config['twitter']['api_key_secret']

access_token = config['twitter']['access_token']
access_token_secret = config['twitter']['access_token_secret']

In [46]:
# authenticate
auth = tweepy.OAuthHandler(api_key, api_key_secret)
auth.set_access_token(access_token, access_token_secret)

In [47]:
api = tweepy.API(auth)

In [50]:
if api.verify_credentials() == False:
    print("The user credentials are invalid.")
else:
    print("The user credentials are valid.")

The user credentials are valid.


In [81]:
page_list = []

for page in tweepy.Cursor(api.user_timeline, screen_name='guardian', count=200).pages(16):
    page_list.append(page)

In [82]:
len(page_list)

16

In [83]:
len(page_list[0])

200

In [84]:
all_statuses_lst = []

for page in page_list:
    for status in page:
        all_statuses_lst.append(status._json)

In [90]:
all_statuses_lst[0].keys()

dict_keys(['created_at', 'id', 'id_str', 'text', 'truncated', 'entities', 'source', 'in_reply_to_status_id', 'in_reply_to_status_id_str', 'in_reply_to_user_id', 'in_reply_to_user_id_str', 'in_reply_to_screen_name', 'user', 'geo', 'coordinates', 'place', 'contributors', 'is_quote_status', 'retweet_count', 'favorite_count', 'favorited', 'retweeted', 'possibly_sensitive', 'lang'])

In [91]:
all_statuses_lst[0]

{'created_at': 'Wed Sep 28 21:17:47 +0000 2022',
 'id': 1575233331943309339,
 'id_str': '1575233331943309339',
 'text': 'Morning mail: hurricane with 240km/h winds hits Florida, blueberry pesticide fears, rents climbing fast https://t.co/lLrbI2KBKy',
 'truncated': False,
 'entities': {'hashtags': [],
  'symbols': [],
  'user_mentions': [],
  'urls': [{'url': 'https://t.co/lLrbI2KBKy',
    'expanded_url': 'https://www.theguardian.com/australia-news/2022/sep/29/morning-mail-hurricane-with-240kmh-winds-hits-florida-blueberry-pesticide-fears-rents-climbing-fast?utm_term=Autofeed&CMP=twt_gu&utm_medium&utm_source=Twitter#Echobox=1664399777',
    'display_url': 'theguardian.com/australia-news…',
    'indices': [104, 127]}]},
 'source': '<a href="https://www.echobox.com" rel="nofollow">Echobox</a>',
 'in_reply_to_status_id': None,
 'in_reply_to_status_id_str': None,
 'in_reply_to_user_id': None,
 'in_reply_to_user_id_str': None,
 'in_reply_to_screen_name': None,
 'user': {'id': 87818409,
  'id

In [102]:
guardian_tweets_df = pd.json_normalize(all_statuses_lst)

In [103]:
cols_to_drop = []

for col in guardian_tweets_df.columns:
    if col.startswith('user'):
        cols_to_drop.append(col)

In [104]:
len(cols_to_drop)

44

In [105]:
for col in guardian_tweets_df.columns:
    if col.startswith('retweeted_status'):
        cols_to_drop.append(col)

In [106]:
len(cols_to_drop)

114

In [107]:
print(guardian_tweets_df.columns)

Index(['created_at', 'id', 'id_str', 'text', 'truncated', 'source',
       'in_reply_to_status_id', 'in_reply_to_status_id_str',
       'in_reply_to_user_id', 'in_reply_to_user_id_str',
       ...
       'retweeted_status.coordinates', 'retweeted_status.place',
       'retweeted_status.contributors', 'retweeted_status.is_quote_status',
       'retweeted_status.retweet_count', 'retweeted_status.favorite_count',
       'retweeted_status.favorited', 'retweeted_status.retweeted',
       'retweeted_status.possibly_sensitive', 'retweeted_status.lang'],
      dtype='object', length=142)


In [108]:
guardian_tweets_df.drop(cols_to_drop, axis=1, inplace=True)

In [109]:
guardian_tweets_df.columns

Index(['created_at', 'id', 'id_str', 'text', 'truncated', 'source',
       'in_reply_to_status_id', 'in_reply_to_status_id_str',
       'in_reply_to_user_id', 'in_reply_to_user_id_str',
       'in_reply_to_screen_name', 'geo', 'coordinates', 'place',
       'contributors', 'is_quote_status', 'retweet_count', 'favorite_count',
       'favorited', 'retweeted', 'possibly_sensitive', 'lang',
       'entities.hashtags', 'entities.symbols', 'entities.user_mentions',
       'entities.urls', 'entities.media', 'extended_entities.media'],
      dtype='object')

In [110]:
guardian_tweets_df.head()

,created_at,id,id_str,text,truncated,source,in_reply_to_status_id,in_reply_to_status_id_str,in_reply_to_user_id,in_reply_to_user_id_str,...,favorited,retweeted,possibly_sensitive,lang,entities.hashtags,entities.symbols,entities.user_mentions,entities.urls,entities.media,extended_entities.media
0,Wed Sep 28 21:17:47 +0000 2022,1575233331943309339,1575233331943309339,Morning mail: hurricane with 240km/h winds hit...,False,"<a href=""https://www.echobox.com"" rel=""nofollo...",NaN,None,NaN,None,...,False,False,False,en,[],[],[],"[{'url': 'https://t.co/lLrbI2KBKy', 'expanded_...",NaN,NaN
1,Wed Sep 28 21:17:46 +0000 2022,1575233327329574924,1575233327329574924,"R Kelly ordered to pay restitution of $300,000...",False,"<a href=""https://www.echobox.com"" rel=""nofollo...",NaN,None,NaN,None,...,False,False,False,en,[],[],[],"[{'url': 'https://t.co/nysKuers5s', 'expanded_...",NaN,NaN
2,Wed Sep 28 21:17:44 +0000 2022,1575233321063292928,1575233321063292928,Two aircraft involved in ‘minor collision’ on ...,False,"<a href=""https://www.echobox.com"" rel=""nofollo...",NaN,None,NaN,None,...,False,False,False,en,[],[],[],"[{'url': 'https://t.co/MgdkAQIptN', 'expanded_...",NaN,NaN
3,Wed Sep 28 21:10:00 +0000 2022,1575231372490309658,1575231372490309658,We’re keen to hear from people who have recent...,True,"<a href=""https://about.twitter.com/products/tw...",NaN,None,NaN,None,...,False,False,False,en,[],[],[],"[{'url': 'https://t.co/BgvOGN3fav', 'expanded_...",NaN,NaN
4,Wed Sep 28 21:03:05 +0000 2022,1575229631912869894,1575229631912869894,"Guardian front page, Thursday 29 September 202...",False,"<a href=""https://about.twitter.com/products/tw...",NaN,None,NaN,None,...,False,False,False,en,[],[],[],[],"[{'id': 1575229403981807628, 'id_str': '157522...","[{'id': 1575229403981807628, 'id_str': '157522..."


In [111]:
guardian_tweets_df.count()

created_at                   3200
id                           3200
id_str                       3200
text                         3200
truncated                    3200
source                       3200
in_reply_to_status_id           1
in_reply_to_status_id_str       1
in_reply_to_user_id             1
in_reply_to_user_id_str         1
in_reply_to_screen_name         1
geo                             0
coordinates                     0
place                           0
contributors                    0
is_quote_status              3200
retweet_count                3200
favorite_count               3200
favorited                    3200
retweeted                    3200
possibly_sensitive           3199
lang                         3200
entities.hashtags            3200
entities.symbols             3200
entities.user_mentions       3200
entities.urls                3200
entities.media                 15
extended_entities.media        15
dtype: int64

In [113]:
for col in guardian_tweets_df.columns:
    if guardian_tweets_df[col].isna().sum() > 0:
        guardian_tweets_df.drop(col, axis=1, inplace=True)

In [114]:
guardian_tweets_df.columns

Index(['created_at', 'id', 'id_str', 'text', 'truncated', 'source',
       'is_quote_status', 'retweet_count', 'favorite_count', 'favorited',
       'retweeted', 'lang', 'entities.hashtags', 'entities.symbols',
       'entities.user_mentions', 'entities.urls'],
      dtype='object')

In [115]:
guardian_tweets_df.head()

,created_at,id,id_str,text,truncated,source,is_quote_status,retweet_count,favorite_count,favorited,retweeted,lang,entities.hashtags,entities.symbols,entities.user_mentions,entities.urls
0,Wed Sep 28 21:17:47 +0000 2022,1575233331943309339,1575233331943309339,Morning mail: hurricane with 240km/h winds hit...,False,"<a href=""https://www.echobox.com"" rel=""nofollo...",False,2,9,False,False,en,[],[],[],"[{'url': 'https://t.co/lLrbI2KBKy', 'expanded_..."
1,Wed Sep 28 21:17:46 +0000 2022,1575233327329574924,1575233327329574924,"R Kelly ordered to pay restitution of $300,000...",False,"<a href=""https://www.echobox.com"" rel=""nofollo...",False,2,13,False,False,en,[],[],[],"[{'url': 'https://t.co/nysKuers5s', 'expanded_..."
2,Wed Sep 28 21:17:44 +0000 2022,1575233321063292928,1575233321063292928,Two aircraft involved in ‘minor collision’ on ...,False,"<a href=""https://www.echobox.com"" rel=""nofollo...",False,6,12,False,False,en,[],[],[],"[{'url': 'https://t.co/MgdkAQIptN', 'expanded_..."
3,Wed Sep 28 21:10:00 +0000 2022,1575231372490309658,1575231372490309658,We’re keen to hear from people who have recent...,True,"<a href=""https://about.twitter.com/products/tw...",False,3,5,False,False,en,[],[],[],"[{'url': 'https://t.co/BgvOGN3fav', 'expanded_..."
4,Wed Sep 28 21:03:05 +0000 2022,1575229631912869894,1575229631912869894,"Guardian front page, Thursday 29 September 202...",False,"<a href=""https://about.twitter.com/products/tw...",False,51,114,False,False,en,[],[],[],[]


In [120]:
os.makedirs('../data/raw/gu_twitter', exist_ok=True)

In [121]:
guardian_tweets_df.to_csv('../data/raw/gu_twitter/gu_timeline.csv', index=False)

In [117]:
tweets_text_df = guardian_tweets_df[['created_at', 'id', 'text']]

In [118]:
tweets_text_df

,created_at,id,text
0,Wed Sep 28 21:17:47 +0000 2022,1575233331943309339,Morning mail: hurricane with 240km/h winds hit...
1,Wed Sep 28 21:17:46 +0000 2022,1575233327329574924,"R Kelly ordered to pay restitution of $300,000..."
2,Wed Sep 28 21:17:44 +0000 2022,1575233321063292928,Two aircraft involved in ‘minor collision’ on ...
3,Wed Sep 28 21:10:00 +0000 2022,1575231372490309658,We’re keen to hear from people who have recent...
4,Wed Sep 28 21:03:05 +0000 2022,1575229631912869894,"Guardian front page, Thursday 29 September 202..."
...,...,...,...
3195,Wed Sep 14 19:57:04 +0000 2022,1570139587145760768,Michelin hands out stars to 13 Toronto restaur...
3196,Wed Sep 14 19:50:43 +0000 2022,1570137992370552840,‘Lie of gender identity’ spurred founding of L...
3197,Wed Sep 14 19:46:17 +0000 2022,1570136874206826496,"End of Covid pandemic ‘in sight’, says World H..."
3198,Wed Sep 14 19:38:56 +0000 2022,1570135026754551808,Fulfilling her destiny of lying in state in We...


In [122]:
tweets_text_df.to_csv('../data/raw/gu_twitter/gu_tweets.csv', index=False)